# Project 3: Crystal Lattice

In [3]:
from vpython import *
import numpy as np

In [4]:
# Constants
N = 3  # Number of atoms per side
k = 5  # Spring constant
mass = 1  # Mass of each atom
damping = 0  # No damping (undamped system)
dt = 0.01  # Time step
L = 1  # Equilibrium bond length

In [5]:
# Hooke's Law force calculations (only between defined pairs)
def spring_force(atomA, atomB, vA, vB, k):
    displacement = atomB.pos - atomA.pos
    stretch = mag(displacement) - mag(atomB.pos - atomA.pos)
    force_magnitude = -k * stretch
    force_direction = norm(displacement)
    force = force_magnitude * force_direction
    return force

In [6]:
def createCube2():
    scene = canvas()
        # Create atoms (spheres) and store them in a dictionary
    atoms = {}
    velocities = {}
    springs = []
    
    for x in range(N):
        for y in range(N):
            for z in range(N):
                pos = vector(x * L, y * L, z * L)
                atom = sphere(pos=pos, radius=0.2, color=color.cyan, make_trail=False)
                atoms[(x, y, z)] = atom
                velocities[(x, y, z)] = vector(np.random.uniform(-1, 1), np.random.uniform(-1, 1), np.random.uniform(-1, 1))
    
    # Function to get neighbors
    neighbors = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]
    
    # Create springs (cylinders) connecting neighboring atoms
    for (x, y, z), atom in atoms.items():
        for dx, dy, dz in neighbors:
            neighbor_key = (x + dx, y + dy, z + dz)
            if neighbor_key in atoms:
                neighbor = atoms[neighbor_key]
                spring = cylinder(pos=atom.pos, axis=neighbor.pos - atom.pos, radius=0.05, color=color.white)
                springs.append((spring, (x, y, z), neighbor_key))
                
    while True:
        rate(100)
        forces = {key: vector(0, 0, 0) for key in atoms}
        for (x, y, z), atom in atoms.items():
            for dx, dy, dz in neighbors:
                neighbor_key = (x + dx, y + dy, z + dz)
                if neighbor_key in atoms:
                    neighbor = atoms[neighbor_key]
                    displacement = neighbor.pos - atom.pos
                    extension = mag(displacement) - L
                    force = k * extension * norm(displacement)
                    forces[(x, y, z)] += force
                    forces[neighbor_key] -= force
        
        # Update positions and velocities
        for key in atoms:
            acceleration = forces[key] / mass
            velocities[key] += acceleration * dt
            atoms[key].pos += velocities[key] * dt
        
        # Update spring positions
        for spring, key1, key2 in springs:
            spring.pos = atoms[key1].pos
            spring.axis = atoms[key2].pos - atoms[key1].pos

In [7]:
def createCube3():
    scene = canvas()

    # Create atoms (spheres) and store them in a dictionary
    atoms = {}
    velocities = {}
    springs = []

    for x in range(N):
        for y in range(N):
            for z in range(N):
                pos = vector(x * L, y * L, z * L)
                atom = sphere(pos=pos, radius=0.2, color=color.cyan, make_trail=False)
                atoms[(x, y, z)] = atom
                velocities[(x, y, z)] = vector(np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1))  # Smaller initial velocity

    # Function to get neighbors (Face-connected + Edge-connected + 3D Diagonal)
    neighbors = [
        (1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1),  # Face-connected (6)
        (1, 1, 0), (-1, -1, 0), (1, -1, 0), (-1, 1, 0),  # Edge-connected (XY plane)
        (1, 0, 1), (-1, 0, -1), (1, 0, -1), (-1, 0, 1),  # Edge-connected (XZ plane)
        (0, 1, 1), (0, -1, -1), (0, 1, -1), (0, -1, 1),  # Edge-connected (YZ plane)
        (1, 1, 1), (-1, -1, -1), (1, 1, -1), (-1, -1, 1),  # 3D Space Diagonal
        (1, -1, 1), (-1, 1, -1), (1, -1, -1), (-1, 1, 1)
    ]

    # Create springs (cylinders) connecting neighboring atoms
    for (x, y, z), atom in atoms.items():
        for dx, dy, dz in neighbors:
            neighbor_key = (x + dx, y + dy, z + dz)
            if neighbor_key in atoms:
                neighbor = atoms[neighbor_key]
                spring = cylinder(pos=atom.pos, axis=neighbor.pos - atom.pos, radius=0.05, color=color.white)
                springs.append((spring, (x, y, z), neighbor_key))

    damping_factor = 0.98  # Reduce velocities over time to prevent infinite oscillations

    while True:
        rate(100)
        forces = {key: vector(0, 0, 0) for key in atoms}

        # **Spring forces**
        for (x, y, z), atom in atoms.items():
            for dx, dy, dz in neighbors:
                neighbor_key = (x + dx, y + dy, z + dz)
                if neighbor_key in atoms:
                    neighbor = atoms[neighbor_key]
                    displacement = neighbor.pos - atom.pos
                    rest_length = mag(vector(dx, dy, dz) * L)  # Rest length depends on diagonal
                    extension = mag(displacement) - rest_length
                    force = k * extension * norm(displacement)
                    forces[(x, y, z)] += force
                    forces[neighbor_key] -= force

        # **Repulsive forces (stronger, nonlinear)**
        keys = list(atoms.keys())
        for i in range(len(keys)):
            for j in range(i + 1, len(keys)):  # Avoid redundant calculations
                key1, key2 = keys[i], keys[j]
                atom1, atom2 = atoms[key1], atoms[key2]

                dist_vec = atom2.pos - atom1.pos
                distance = mag(dist_vec)
                min_dist = 0.4  # Prevent overlap

                if distance < min_dist:
                    # Use a strong exponential repulsion force
                    repulsion_strength = 500 * exp(-distance / 0.1)
                    repulsion_force = repulsion_strength * norm(dist_vec)

                    forces[key1] -= repulsion_force
                    forces[key2] += repulsion_force

                    # Adjust velocities to prevent overlap
                    velocities[key1] -= norm(repulsion_force) * dt
                    velocities[key2] += norm(repulsion_force) * dt

        # **Update positions and velocities**
        for key in atoms:
            acceleration = forces[key] / mass
            velocities[key] = velocities[key] * damping_factor + acceleration * dt  # Apply damping
            atoms[key].pos += velocities[key] * dt

        # **Update spring positions**
        for spring, key1, key2 in springs:
            spring.pos = atoms[key1].pos
            spring.axis = atoms[key2].pos - atoms[key1].pos


In [8]:
x = createCube2()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [ ]:
x = createCube3()

In [8]:
def createCube4():
    scene = canvas()

    # Create atoms (spheres) and store them in a dictionary
    atoms = {}
    velocities = {}
    springs = []

    for x in range(N):
        for y in range(N):
            for z in range(N):
                pos = vector(x * L, y * L, z * L)
                atom = sphere(pos=pos, radius=0.2, color=color.cyan, make_trail=False)
                atoms[(x, y, z)] = atom
                velocities[(x, y, z)] = vector(np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1))  # Smaller initial velocity

    # Function to get neighbors (Face-connected)
    face_neighbors = [
        (1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)
    ]

    # Diagonal neighbors (only for outer atoms)
    diagonal_neighbors = [
        (1, 1, 0), (-1, -1, 0), (1, -1, 0), (-1, 1, 0),  # Edge-connected (XY)
        (1, 0, 1), (-1, 0, -1), (1, 0, -1), (-1, 0, 1),  # Edge-connected (XZ)
        (0, 1, 1), (0, -1, -1), (0, 1, -1), (0, -1, 1),  # Edge-connected (YZ)
        (1, 1, 1), (-1, -1, -1), (1, 1, -1), (-1, -1, 1),  # 3D Space Diagonal
        (1, -1, 1), (-1, 1, -1), (1, -1, -1), (-1, 1, 1)
    ]

    # Create springs (cylinders) connecting neighboring atoms
    for (x, y, z), atom in atoms.items():
        is_outer = (x == 0 or x == N - 1 or y == 0 or y == N - 1 or z == 0 or z == N - 1)

        # Add face-connected springs (always)
        for dx, dy, dz in face_neighbors:
            neighbor_key = (x + dx, y + dy, z + dz)
            if neighbor_key in atoms:
                neighbor = atoms[neighbor_key]
                spring = helix(pos=atom.pos, axis=neighbor.pos - atom.pos, radius=0.05, color=color.white)
                springs.append((spring, (x, y, z), neighbor_key))

        # Add diagonal springs only if this atom is on the outer surface
        if is_outer:
            for dx, dy, dz in diagonal_neighbors:
                neighbor_key = (x + dx, y + dy, z + dz)
                if neighbor_key in atoms:
                    neighbor = atoms[neighbor_key]
                    spring = cylinder(pos=atom.pos, axis=neighbor.pos - atom.pos, radius=0.05, color=color.red)  # Diagonal springs in red
                    springs.append((spring, (x, y, z), neighbor_key))

    damping_factor = 0.98  # Reduce velocities over time to prevent infinite oscillations

    while True:
        rate(100)
        forces = {key: vector(0, 0, 0) for key in atoms}

        # **Spring forces**
        for (x, y, z), atom in atoms.items():
            is_outer = (x == 0 or x == N - 1 or y == 0 or y == N - 1 or z == 0 or z == N - 1)

            # Apply forces for face-connected springs (always)
            for dx, dy, dz in face_neighbors:
                neighbor_key = (x + dx, y + dy, z + dz)
                if neighbor_key in atoms:
                    neighbor = atoms[neighbor_key]
                    displacement = neighbor.pos - atom.pos
                    extension = mag(displacement) - L
                    force = k * extension * norm(displacement)
                    forces[(x, y, z)] += force
                    forces[neighbor_key] -= force

            # Apply forces for diagonal springs only if on the outer surface
            if is_outer:
                for dx, dy, dz in diagonal_neighbors:
                    neighbor_key = (x + dx, y + dy, z + dz)
                    if neighbor_key in atoms:
                        neighbor = atoms[neighbor_key]
                        displacement = neighbor.pos - atom.pos
                        rest_length = mag(vector(dx, dy, dz) * L)
                        extension = mag(displacement) - rest_length
                        force = k * extension * norm(displacement)
                        forces[(x, y, z)] += force
                        forces[neighbor_key] -= force

        # **Repulsive forces (stronger, nonlinear)**
        keys = list(atoms.keys())
        for i in range(len(keys)):
            for j in range(i + 1, len(keys)):  # Avoid redundant calculations
                key1, key2 = keys[i], keys[j]
                atom1, atom2 = atoms[key1], atoms[key2]

                dist_vec = atom2.pos - atom1.pos
                distance = mag(dist_vec)
                min_dist = 0.4  # Prevent overlap

                if distance < min_dist:
                    # Use a strong exponential repulsion force
                    repulsion_strength = 500 * exp(-distance / 0.1)
                    repulsion_force = repulsion_strength * norm(dist_vec)

                    forces[key1] -= repulsion_force
                    forces[key2] += repulsion_force

                    # Adjust velocities to prevent overlap
                    velocities[key1] -= norm(repulsion_force) * dt
                    velocities[key2] += norm(repulsion_force) * dt

        # **Update positions and velocities**
        for key in atoms:
            acceleration = forces[key] / mass
            velocities[key] = velocities[key] * damping_factor + acceleration * dt  # Apply damping
            atoms[key].pos += velocities[key] * dt

        # **Update spring positions**
        for spring, key1, key2 in springs:
            spring.pos = atoms[key1].pos
            spring.axis = atoms[key2].pos - atoms[key1].pos


In [ ]:
x = createCube4()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Define parameters
n = 3  # Size of the crystal (3x3x3)
k = 10  # Spring constant
mass = 1  # Mass of each sphere
dt = 0.01  # Time step for simulation
spring_length = 1.0  # Natural length of the springs

# Create a 3x3x3 matrix of spheres
spheres = np.empty((n, n, n), dtype=object)
positions = np.zeros((n, n, n, 3))  # Array to store positions
velocities = np.random.rand(n, n, n, 3) - 0.5  # Random initial velocities

# Create spheres and place them in the 3x3x3 grid
for i in range(n):
    for j in range(n):
        for k in range(n):
            pos = np.array([i, j, k]) * spring_length
            spheres[i, j, k] = sphere(pos=vector(*pos), radius=0.2, color=color.cyan)
            positions[i, j, k] = pos